# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Part 3 of the assigment

#### Importing Libraries

In [1]:
import numpy as np #Numpy Library
import pandas as pd #Panda Library

In [2]:
from bs4 import BeautifulSoup #BeautifulSoup to read the html

In [3]:
import requests #to request the page

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [5]:
!conda install -c conda-forge geocoder
import geocoder

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge


#### PART 1 of the assigment

In [6]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
soup = BeautifulSoup(source,'lxml')

In [8]:
tabla = soup.find('table', attrs={'class':'wikitable sortable'})
#print(tabla.prettify())  #prettify to display in a more friendly way

In [9]:
filas = tabla.find_all('tr')
l=[]
for tr in filas:
    td = tr.find_all('td')
    fila = [tr.text for tr in td]
    l.append(fila)
lista = pd.DataFrame(l, columns=['PostCode','Borough','Neigborhood'])

In [10]:
lista = lista.replace('\n','', regex=True)

In [11]:
lista.drop(lista.index[0],inplace=True)

In [12]:
lista = lista[lista.Borough != 'Not assigned']

In [13]:
lista['Neigborhood']= np.where(lista['Neigborhood']=='Not assigned',lista['Borough'],lista['Neigborhood'])

In [14]:
lista.sort_values('PostCode',inplace=True)

In [15]:
concatenar = {
    'Neigborhood': lambda x: ', '.join(x)
}

In [16]:
lista = lista.groupby(['PostCode','Borough'], as_index=False, sort=False).agg(concatenar)

## Part 2 of the assigment

Reading the csv file with the geospatial data for the postal codes that the geocoder returns error

In [17]:
gd=pd.read_csv('https://cocl.us/Geospatial_data',index_col=0)
print('Data loaded')

Data loaded


In [18]:
gd.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [19]:
# The code was removed by Watson Studio for sharing.

Using the geocoder to obtain the latitude and longitude of the Postal Code

In [20]:
lista['Latitude']=0
lista['Longitude']=0
for i in range(0, len(lista)):
    ubicacion ='\''+lista['Borough'].iloc[i]+','+lista['PostCode'].iloc[i]+'\''
    try:
        g=geocoder.here(ubicacion,app_id=ID,app_code=CODE) # I'm using the geocoder HERE
        #print(i,"-",ubicacion,':',g.lat,' ',g.lng)
        lista['Latitude'].iloc[i] = g.lat
        lista['Longitude'].iloc[i]= g.lng
    except: #If the geocoder returns an error, I use gd dataframe to look for the latitude and longitude of the postal code
        postalcode=lista['PostCode'].iloc[i]
        lista['Latitude'].iloc[i] = gd['Latitude'].loc[postalcode]
        lista['Longitude'].iloc[i]= gd['Longitude'].loc[postalcode]
        #print(i,"-",ubicacion,':', lista['Latitude'].iloc[i],' ',lista['Longitude'].iloc[i])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# Part 3 of the assigment

#### For this part of the assigment, i'm going to replicate the same analysis we did to the New York City data in the last lab

In [21]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge


First, we need to obtain how many Boroughs and Neighborhoods we have

In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(lista['Borough'].unique()),
        lista.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


I'm going to display in a map the different neighborhoods, to center the map, I'm going to obtain the latitude and longitude of Toronto

In [23]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.7170226, -79.4197830350134.


In [24]:
# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(lista['Latitude'], lista['Longitude'], lista['Borough'], lista['Neigborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

I'm going to choose a borough to work with it

In [25]:
lista.groupby(['Borough']).size()

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
dtype: int64

I'm going to filter by the Etobicoke borough who has 12 neighborhoods

In [26]:
etobicoke_data = lista[lista['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head()

,PostCode,Borough,Neigborhood,Latitude,Longitude
0,M8V,Etobicoke,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818
1,M8W,Etobicoke,"Long Branch, Alderwood",43.60113,-79.53878
2,M8X,Etobicoke,"Old Mill North, Montgomery Road, The Kingsway",43.65369,-79.51112
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Humber Bay, ...",43.63277,-79.48960
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.62463,-79.52695


Let see in a map the different Neighborhoods of Etobike

In [27]:
# we obtain the coordinates of Etobicoke, Canada
address = 'Etobicoke, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [28]:
# create map of Etobicoke using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neigborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [29]:
# The code was removed by Watson Studio for sharing.

#### Lets explore the first neighborhood

In [30]:
etobicoke_data.loc[0, 'Neigborhood']

'Humber Bay Shores, New Toronto, Mimico South'

In [31]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neigborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Humber Bay Shores, New Toronto, Mimico South are 43.60987, -79.49818.


Now we are going to obtain the maximum 100 venues in a radius of 500 of the first neighborhood 

In [32]:
radius = 500
LIMIT = 100
# First we have to form the URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c2e61934c1f6766005330d3'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e88ebbb8231210e528f1523-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '4e88ebbb8231210e528f1523',
       'location': {'address': '220 Royal York Rd',
        'cc': 'CA',
        'city': 'Etobicoke',
        'country': 'Canada',
        'crossStreet': 'Mimico Ave.',
        'distance': 287,
        'formattedAddress': ['220 Royal York Rd (Mimico Ave.)',
         'Etobicoke ON M8V 2V7',
         'Canada'],
        

We are going to use the following funtion to extract the category of the venue

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mario & Selina's No Frills,Grocery Store,43.612340,-79.497148
1,Mimico Arena,Skating Rink,43.612739,-79.498682
2,Hasty Market,Convenience Store,43.611390,-79.496398
3,Hillside park,Park,43.610296,-79.494541
4,Bikram Red Tree,Yoga Studio,43.612906,-79.498282


so, for the fist neighborhood of Etobicoke we have 

In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


Now, we define the following funtion to obtain the venues of all the neighborhoods of Etobicoke

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We create a pandas dataframe with the near venues of every neighborhood

In [38]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neigborhood'], latitudes=etobicoke_data['Latitude'], longitudes=etobicoke_data['Longitude'])

Humber Bay Shores, New Toronto, Mimico South
Long Branch, Alderwood
Old Mill North, Montgomery Road, The Kingsway
Old Mill South, King's Mill Park, Humber Bay, The Queensway East, Royal York South East, Sunnylea, Mimico NE, Kingsway Park South East
Kingsway Park South West, Mimico NW, The Queensway West, South of Bloor, Royal York South West
Islington Avenue
Cloverdale, West Deane Park, Princess Gardens, Martin Grove, Islington
Old Burnhamthorpe, Markland Wood, Eringate, Bloordale Gardens
Westmount
Richview Gardens, Kingsview Village, St. Phillips, Martin Grove Gardens
Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, Thistletown, Albion Gardens, South Steeles
Northwest


Let's check the size of the resulting dataframe

In [39]:
print(etobicoke_venues.shape)
etobicoke_venues.head()

(88, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818,Mario & Selina's No Frills,43.612340,-79.497148,Grocery Store
1,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818,Mimico Arena,43.612739,-79.498682,Skating Rink
2,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818,Hasty Market,43.611390,-79.496398,Convenience Store
3,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818,Hillside park,43.610296,-79.494541,Park
4,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818,Bikram Red Tree,43.612906,-79.498282,Yoga Studio


and how many venues were returned for each neighborhood

In [40]:
etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, Thistletown, Albion Gardens, South Steeles",14,14,14,14,14,14
"Cloverdale, West Deane Park, Princess Gardens, Martin Grove, Islington",6,6,6,6,6,6
"Humber Bay Shores, New Toronto, Mimico South",6,6,6,6,6,6
Islington Avenue,8,8,8,8,8,8
"Kingsway Park South West, Mimico NW, The Queensway West, South of Bloor, Royal York South West",17,17,17,17,17,17
"Long Branch, Alderwood",5,5,5,5,5,5
Northwest,6,6,6,6,6,6
"Old Burnhamthorpe, Markland Wood, Eringate, Bloordale Gardens",6,6,6,6,6,6
"Old Mill North, Montgomery Road, The Kingsway",3,3,3,3,3,3


and how many categories we have

In [41]:
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 50 uniques categories.


#### Analyze Each Neighborhood

In [42]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighborhood,Arts & Crafts Store,Asian Restaurant,Bank,Beer Store,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,...,Pub,Sandwich Place,Shopping Mall,Skating Rink,Storage Facility,Sushi Restaurant,Tea Room,Thai Restaurant,Video Store,Yoga Studio
0,"Humber Bay Shores, New Toronto, Mimico South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Humber Bay Shores, New Toronto, Mimico South",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,"Humber Bay Shores, New Toronto, Mimico South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Humber Bay Shores, New Toronto, Mimico South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Humber Bay Shores, New Toronto, Mimico South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


lets check the size of the dataframe

In [43]:
etobicoke_onehot.shape

(88, 51)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,Arts & Crafts Store,Asian Restaurant,Bank,Beer Store,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,...,Pub,Sandwich Place,Shopping Mall,Skating Rink,Storage Facility,Sushi Restaurant,Tea Room,Thai Restaurant,Video Store,Yoga Studio
0,"Beaumond Heights, Humbergate, Jamestown, Mount...",0.00,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.071429,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000
1,"Cloverdale, West Deane Park, Princess Gardens,...",0.00,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.166667,0.000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000
2,"Humber Bay Shores, New Toronto, Mimico South",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.000000,0.000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667
3,Islington Avenue,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.000000,0.125,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Kingsway Park South West, Mimico NW, The Queen...",0.00,0.058824,0.058824,0.000000,0.058824,0.058824,0.176471,0.00,0.00,...,0.0,0.000000,0.000,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000
5,"Long Branch, Alderwood",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.2,0.200000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Northwest,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.00,...,0.0,0.166667,0.000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Old Burnhamthorpe, Markland Wood, Eringate, Bl...",0.00,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Old Mill North, Montgomery Road, The Kingsway",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Old Mill South, King's Mill Park, Humber Bay, ...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,...,0.0,0.000000,0.000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000


We have 12 neighborhoods and 51 categories

In [45]:
etobicoke_grouped.shape

(12, 51)

Let's print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, Thistletown, Albion Gardens, South Steeles----
                 venue  freq
0        Grocery Store  0.14
1          Pizza Place  0.07
2       Sandwich Place  0.07
3  Japanese Restaurant  0.07
4       Hardware Store  0.07


----Cloverdale, West Deane Park, Princess Gardens, Martin Grove, Islington----
                venue  freq
0         Pizza Place  0.33
1                Bank  0.17
2            Tea Room  0.17
3      Sandwich Place  0.17
4  Chinese Restaurant  0.17


----Humber Bay Shores, New Toronto, Mimico South----
               venue  freq
0               Park  0.33
1        Yoga Studio  0.17
2      Grocery Store  0.17
3       Skating Rink  0.17
4  Convenience Store  0.17


----Islington Avenue----
           venue  freq
0       Pharmacy  0.25
1           Bank  0.12
2  Grocery Store  0.12
3   Skating Rink  0.12
4           Café  0.12


----Kingsway Park South West, Mimico NW, The Queensway West, South of Bloor

Now, we are going  to create the new dataframe and display the top 10 venues for each neighborhood

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Beaumond Heights, Humbergate, Jamestown, Mount...",Grocery Store,Pizza Place,Fast Food Restaurant,Video Store,Japanese Restaurant,Liquor Store,Coffee Shop,Fried Chicken Joint,Pharmacy,Hardware Store
1,"Cloverdale, West Deane Park, Princess Gardens,...",Pizza Place,Tea Room,Bank,Sandwich Place,Chinese Restaurant,Yoga Studio,Carpet Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant
2,"Humber Bay Shores, New Toronto, Mimico South",Park,Yoga Studio,Grocery Store,Skating Rink,Convenience Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant
3,Islington Avenue,Pharmacy,Café,Skating Rink,Park,Shopping Mall,Grocery Store,Bank,Buffet,Chinese Restaurant,Fried Chicken Joint
4,"Kingsway Park South West, Mimico NW, The Queen...",Burrito Place,Gym / Fitness Center,Burger Joint,Middle Eastern Restaurant,Miscellaneous Shop,Optical Shop,Eastern European Restaurant,Coffee Shop,Mattress Store,Buffet
5,"Long Branch, Alderwood",Performing Arts Venue,Sandwich Place,Pub,Gym,Candy Store,Carpet Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
6,Northwest,Storage Facility,Burger Joint,Sandwich Place,Middle Eastern Restaurant,Gym,Coffee Shop,Yoga Studio,Chinese Restaurant,Fried Chicken Joint,Fish & Chips Shop
7,"Old Burnhamthorpe, Markland Wood, Eringate, Bl...",Carpet Store,Bank,Grocery Store,Fish & Chips Shop,Electronics Store,College Rec Center,Chinese Restaurant,Gym,Fried Chicken Joint,Fast Food Restaurant
8,"Old Mill North, Montgomery Road, The Kingsway",Park,Lounge,Pool,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Convenience Store
9,"Old Mill South, King's Mill Park, Humber Bay, ...",Park,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Electronics Store


#### Cluster Neighborhoods
> Run *k*-means to cluster the neighborhood into 5 clusters.

In [49]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 1, 1, 1, 0, 3, 4], dtype=int32)

> Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
etobicoke_merged = etobicoke_data

# add clustering labels
etobicoke_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neigborhood')

etobicoke_merged.head() # check the last columns!

,PostCode,Borough,Neigborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"Humber Bay Shores, New Toronto, Mimico South",43.60987,-79.49818,0,Park,Yoga Studio,Grocery Store,Skating Rink,Convenience Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant
1,M8W,Etobicoke,"Long Branch, Alderwood",43.60113,-79.53878,4,Performing Arts Venue,Sandwich Place,Pub,Gym,Candy Store,Carpet Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
2,M8X,Etobicoke,"Old Mill North, Montgomery Road, The Kingsway",43.65369,-79.51112,0,Park,Lounge,Pool,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Convenience Store
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Humber Bay, ...",43.63277,-79.48960,0,Park,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Electronics Store
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.62463,-79.52695,1,Burrito Place,Gym / Fitness Center,Burger Joint,Middle Eastern Restaurant,Miscellaneous Shop,Optical Shop,Eastern European Restaurant,Coffee Shop,Mattress Store,Buffet


> Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neigborhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The Park cluster:

In [52]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 0, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,0,Park,Yoga Studio,Grocery Store,Skating Rink,Convenience Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant
2,Etobicoke,0,Park,Lounge,Pool,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Eastern European Restaurant,Convenience Store
3,Etobicoke,0,Park,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Electronics Store
7,Etobicoke,0,Carpet Store,Bank,Grocery Store,Fish & Chips Shop,Electronics Store,College Rec Center,Chinese Restaurant,Gym,Fried Chicken Joint,Fast Food Restaurant


This cluster has different categories, I can't assign a name to this cluster

In [53]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 1, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Etobicoke,1,Burrito Place,Gym / Fitness Center,Burger Joint,Middle Eastern Restaurant,Miscellaneous Shop,Optical Shop,Eastern European Restaurant,Coffee Shop,Mattress Store,Buffet
5,Etobicoke,1,Pharmacy,Café,Skating Rink,Park,Shopping Mall,Grocery Store,Bank,Buffet,Chinese Restaurant,Fried Chicken Joint
6,Etobicoke,1,Pizza Place,Tea Room,Bank,Sandwich Place,Chinese Restaurant,Yoga Studio,Carpet Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant


The grocery store cluster

In [54]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 2, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Etobicoke,2,Grocery Store,Pizza Place,Fast Food Restaurant,Video Store,Japanese Restaurant,Liquor Store,Coffee Shop,Fried Chicken Joint,Pharmacy,Hardware Store


The pizza place cluster

In [55]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 3, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Etobicoke,3,Pizza Place,Chinese Restaurant,Intersection,Sandwich Place,Middle Eastern Restaurant,Coffee Shop,Yoga Studio,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant


The arts cluster

In [56]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 4, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Etobicoke,4,Performing Arts Venue,Sandwich Place,Pub,Gym,Candy Store,Carpet Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
9,Etobicoke,4,Arts & Crafts Store,Bus Line,Bus Stop,Music Venue,Coffee Shop,Grocery Store,Fried Chicken Joint,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
11,Etobicoke,4,Storage Facility,Burger Joint,Sandwich Place,Middle Eastern Restaurant,Gym,Coffee Shop,Yoga Studio,Chinese Restaurant,Fried Chicken Joint,Fish & Chips Shop
